Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [27]:
from random import choices, randint, random
from dataclasses import dataclass
import numpy as np
import math
from copy import deepcopy
import lab9_lib

In [2]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

01110100111001001000110100100011110000100000010010: 7.33%
11010101100010101010001001001101110100110111000110: 9.13%
10110110110101000100010001010001100011001001111100: 7.33%
10010111000001011000001011110100110010111100111000: 9.14%
00011100010110011110110101010100111111010010100111: 23.33%
00001010111101001011101010110101001110111000101010: 9.11%
10000000010101011001101110101101101011100111100011: 7.33%
00011100110001111011110101010110010110010010000010: 23.36%
00110101011111001101010110100000001111011111000100: 15.33%
00001001000010010101001111100100000011110011010101: 15.34%
10


A class to describe a single element

Some constants

In [55]:
GENOME_LENGTH = 1_000
TOURNAMENT_SIZE = 3
NUM_EPOCH = 100
OFFSPRING_SIZE = 100
STARTING_SIZE = 10
PROBLEM_SIZE = [1,2,5,10]

A class to contains all the population every single individual of the population is an instance of the class Element

In [4]:
@dataclass
class Element:
    def __init__(self, gen) -> None:
        self.gen = gen
        self.fit = fitness(gen)


In [5]:
@dataclass
class Population:
    def __init__(self, size_starting_population: int):
        self.population = []
        for _ in range(size_starting_population):
            gen = choices([0, 1], k=GENOME_LENGTH)  # K = 1000
            self.population.append(Element(gen))

    def add_offspring(self, offspring):
        self.population.append(offspring)

    def scale_down_population(self, size):
        self.population.sort(key=lambda e: e.fit, reverse=True)
        self.population = self.population[:size]

    def pickone_random(self) -> Element:
        rand_int = randint(0, len(self.population) - 1)
        return self.population[rand_int]

GA (only works with problem size 1-2)
Crossover and mutation
I play a 3 people tournament to select the parents and than I perform a little mutation on the offspring

In [6]:
def tournament(population: Population, tournament_size=1) -> Element:
    best_elem = None
    for _ in range(tournament_size):
        elem = population.pickone_random()
        if best_elem == None:
            best_elem = elem
        if best_elem.fit < elem.fit:
            best_elem = elem
    return best_elem


def mutate(element: Element, num_of_changes=1) -> Element:
    """I've implemented the possiibilty to mutete more than one bit"""
    for _ in range(num_of_changes):
        rand_index = randint(0, GENOME_LENGTH-1)
        element.gen[rand_index] = 1 - element.gen[rand_index]
    return element

def one_cut_crossover(parents) -> Element:
    """function to do one cut crossover"""
    rand_int = randint(0, GENOME_LENGTH-1)
    new_gen = parents[0].gen[rand_int:] + parents[1].gen[:rand_int]
    return Element(new_gen)

def crossover(parents) -> Element:
    """classic crossover"""
    new_gen = []
    for i in range(GENOME_LENGTH):
        new_gen.append(parents[i%2].gen[i])
    return Element(new_gen)

def random_crossover(parents) -> Element:
    """crossover where every genome is pick random from one parent DOESN'T WORK"""
    new_gen = []
    for i in range(GENOME_LENGTH):
        new_gen.append(parents[choices([0,1],k=1).pop()].gen[i])
    return Element(new_gen)

In [56]:
for problem_size in PROBLEM_SIZE:
    # define the population
    population = Population(STARTING_SIZE)

    # start the problem
    fitness = lab9_lib.make_problem(problem_size)

    for _ in range(NUM_EPOCH):
        for _ in range(OFFSPRING_SIZE):
            parent1 = tournament(population, TOURNAMENT_SIZE)
            parent2 = tournament(population, TOURNAMENT_SIZE)
            elem = crossover([parent1,parent2])
            # mutation
            if random() < 0.5:
                elem = mutate(elem)
            population.add_offspring(elem)
        #reduce the population
        population.scale_down_population(OFFSPRING_SIZE)
        if population.population[0].fit == 1:
            break


    population.scale_down_population(1)
    print(f"Problem size {problem_size} -> The best result found is : {population.population.pop().fit} with {fitness.calls} fitness calls")

Problem size 1 -> The best result found is : 0.663 with 10000 fitness calls
Problem size 2 -> The best result found is : 0.538 with 10000 fitness calls
Problem size 5 -> The best result found is : 0.31053 with 10000 fitness calls
Problem size 10 -> The best result found is : 0.204155 with 10000 fitness calls


ES now I try to mutate a lot of bits every single time, I start with more or less 100 bit mutation every time I call mutate()

In [29]:
@dataclass
class Individual:
    def __init__(self,elem :Element, sigma :float, loci :int, prob :float):
        self.elem = elem
        self.loci = loci
        self.prob = prob
        self.sigma = sigma
        
    def mutate(self):
        """mutate following a guassian"""
        if self.elem.fit < 0.8: #the fitness should go from 0 to 1 so I stop the update around 0.8 because we are near the goal
             self.sigma *= np.e**((1/32) * np.random.normal(0,1))
             self.loci *= math.ceil(np.random.normal(1,self.sigma))
             self.prob = np.random.normal(0.5, self.sigma)#it has mu 0.5 because it can be equally 0 or 1
             if self.prob < 0:
                 self.prob = 0
             if self.prob > 1:
                self.prob = 1 
        #actual mutation
        for _ in range(self.loci):
            rand_index = randint(0, GENOME_LENGTH-1)
            if random() < self.prob:
                self.elem.gen[rand_index] = 0
            else:
                self.elem.gen[rand_index] = 1

In [49]:
@dataclass
class PopulationES:
    def __init__(self, size_starting_population: int):
        self.population = []
        for _ in range(size_starting_population):
            gen = choices([0, 1], k=GENOME_LENGTH)  
            self.population.append(Individual(Element(gen), 1, 100, 0 ))# sigma loci prob of 1 or 0

    def add_offspring(self, offspring):
        self.population.append(offspring)

    def scale_down_population(self, size):
        self.population.sort(key=lambda e: e.elem.fit, reverse=True)
        self.population = self.population[:size]

    def pickone_random(self) -> Individual:
        rand_int = randint(0, len(self.population) - 1)
        return self.population[rand_int]

In [57]:
for problem_size in PROBLEM_SIZE:
    # define the population
    population = PopulationES(STARTING_SIZE)

    # start the problem
    fitness = lab9_lib.make_problem(problem_size)

    for _ in range(NUM_EPOCH):
        for _ in range(OFFSPRING_SIZE):
                parent1 = population.pickone_random()
                elem = Individual(parent1.elem,deepcopy(parent1.sigma),deepcopy(parent1.loci),deepcopy(parent1.prob))
                if random() < 0.5:
                    parent2 = population.pickone_random()
                    new_elem = crossover([parent1.elem,parent2.elem])
                    elem = Individual(new_elem, deepcopy(parent1.sigma), deepcopy(parent1.loci), deepcopy(parent1.prob))
                # mutation
                elem.mutate()
                population.add_offspring(elem)
        #reduce the population
        population.scale_down_population(OFFSPRING_SIZE)
        #stop if the goal is reached
        if population.population[0].elem.fit == 1:
            break


    population.scale_down_population(1)
    print(f"Problem size {problem_size} -> The best result found is : {population.population.pop().elem.fit} with {fitness.calls} fitness calls")

Problem size 1 -> The best result found is : 1.0 with 450 fitness calls
Problem size 2 -> The best result found is : 1.0 with 198 fitness calls
Problem size 5 -> The best result found is : 1.0 with 659 fitness calls
Problem size 10 -> The best result found is : 1.0 with 299 fitness calls
